In [91]:
import getpass
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import requests
import os

print('Modules Loaded')

Modules Loaded


In [92]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    # prompt the user to enter their AGOL username
    username = input("Enter your ArcGIS Online username:")
    
    # prompt the user to enter their AGOL password (without echoing)
    password = getpass.getpass("Enter your ArcGIS Onine password:")
    
    # create a GIS object and login to AGOL
    gis = GIS("https://www.arcgis.com", username, password)
    
    # return the object
    return gis

# run module
gis = login_AGOL()

# confirm user login 
print("Logged in as:", gis.users.me.username)

Enter your ArcGIS Online username:Stacey_Vernon_GES23
Enter your ArcGIS Onine password:········
Logged in as: Stacey_Vernon_GES23


In [93]:
# Utilise the AGOL API and the a "static" GeoJSON file to publish data to a feature service. Specifically publish
# Dublin Bike zone data.
# documentation on using the ArcGIS API to create a feature service from GeoJSON here: https://developers.arcgis.com/python/guide/import-data/
# We will determine if a feature layer exists, if not we will create one but if it does - we skip this function

# generate a list of the below search conditions in our AGOL Item Content
zoneSearch = gis.content.search("title:BikeStations_Zones",item_type="Feature Layer")

# if list is empty, create a new feature layer
if not zoneSearch:

# set up the feature service properties in AGOL for the stations and zone dataset
    item_properties = {
    "title": "BikeStations_Zones", #provide a title for your item
    "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }

    # URL link to the GeoJSON file
    geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
    geojson_item = gis.content.add(item_properties, geojson_url)

    # publish the GeoJSON as a feature layer
    BikeStationZone_data = geojson_item.publish()
    print("Data published. New feature layer created.")
else:
    print("Bike Station and Zone data already exists in AGOL.")

Bike Station and Zone data already exists in AGOL.


In [94]:
# Dublin Bike locations are available as a live dataset, updated every 5 minutes
# pull in the Dublin Bike locations from the API

# URL to API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/bikes/bleeper_bikes/current/bikes.geojson"

# make a request to the API
response = requests.get(bike_url)

# if the request was successful (status code 201), get the data
if response.status_code == 200: 
    # parse the JSON response
    bikeData = response.json() 
    
    print ('Success')
else:
    bikeData = None
    print ("Unable to retrieve data. Please run cell again.")
    

Success


In [95]:
# Ensure bikeData is populated
if bikeData is not None:
    # print the data struc
    print(type(bikeData))
    print(bikeData)
    
if 'data' in bikeData:
    bike_details_list = bikeData['data']

<class 'dict'>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-6.233377933502197, 53.3500862121582]}, 'properties': {'system_id': 'bleeper_bikes', 'last_updated': 1722167233, 'bike_id': '0099b5ac-4975-4d24-8731-96eadfc6c2b1', 'is_reserved': False, 'is_disabled': False, 'vehicle_type_id': '79ab8c6d-5498-4ec7-8fdc-efbfedc055ed', 'station_id': '', 'current_fuel_percent': 95, 'current_range_meters': None, 'last_updated_dt': '2024-07-28 11:47:13'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-6.298511981964111, 53.39028549194336]}, 'properties': {'system_id': 'bleeper_bikes', 'last_updated': 1722167233, 'bike_id': '00f50979-0425-4037-aa33-db9ade9025a1', 'is_reserved': False, 'is_disabled': False, 'vehicle_type_id': '79ab8c6d-5498-4ec7-8fdc-efbfedc055ed', 'station_id': '', 'current_fuel_percent': 96, 'current_range_meters': None, 'last_updated_dt': '2024-07-28 11:47:13'}}, {'type': 'Feature', 'geometry': {'t

In [97]:
# Inspect the data structure of bikeData
if bikeData is not None and isinstance(bikeData, list):
    for i, bike in enumerate(bikeData[:5]):  # Display first 5 items for a quick check
        print(f"bikeData[{i}]: {bike}")
else:
    print("bikeData is either None or not a list.")


bikeData is either None or not a list.


In [96]:
# Construct GeoJSON from live data
if bikeData is not None:
    features = []
    for bike_details in bikeData:
        # get required geometry details
        lon = bike_details['lon']
        lat = bike_details['lat']

        # create a dictionary for geometry
        coordinates = [lon,lat] # coordinates must be in long, lat format for GeoJSON
        geometry = {"type":"Point",
                   "coordinates":coordinates}

        # create a dictionary of attributes & geometry
        data_format = {"type":"Feature",
                      "properties": bike_details,
                      "geometry":geometry}

        # add the dictionary to our list
        features.append(data_format)

    # Construct feature layer collection dictionary
    geojson_data = {"type":"FeatureCollection",
                   "features":features}

    # save as geojson
    # get our current working directory (wd)
    wd = os.getcwd()
    file_name = "liveBikeLocation.geojson"
    file_path = os.path.join(wd, file_name)

    with open(file_path, "w") as outfile:
            json.dump(geojson_data, outfile)

    print ("Successfully constructed")
else: 
    print ("No data available to process.")

TypeError: string indices must be integers

In [ ]:
# create a feature layer on AGOL for the live bike data, if it does not already exist
# if it does exist, overwrite it with the new dataset 

#create the FS

item_properities = {
    "title": "Live"
}

In [ ]:
# need to do:
# improve commenting 
# add in docstrings
# publish live bike data
# get live bike data to refresh
# display in webmap
# product heatmap of current locations - deduce busy areas??